# Codigo para geração do indice invertido

In [2]:
import pandas as pd
import math
import nltk
from nltk import bigrams 
import numpy as np
import string
from scipy import sparse
import scipy.sparse as sps

In [36]:
class WordInfo:
    'Classe que guarda informações sobre a palavra, como documentos em que ocorre, tf em cada documento e calculo de idf'

    def __init__(self, word) : # Contrutor da classe recebe apenas a palavra em questão
        self.word = word
        self.idf = 0
        self.docs = {} # Dicionario que mapeia doc_id ao tf da palavra

    def found(self, doc_id) : # Metodo que realiza a contagem do tf da palavra
        if(doc_id in self.docs) : # Se o doc_id está mapeado, incrementa-o
            self.docs[doc_id] += 1
        else :
            self.docs[doc_id] = 1 # Caso contrario, define-o como 1
            
    def calculateIDF(self, totaldocs) : # Metodo de calculo do idf, recebendo o total de documentos
        df = len(self.docs) # Numero de documentos em que a palavra ocorre 
        if (df > 0) :
            self.idf = math.log(((totaldocs + 1)/df))
            
    def getIds(self) : # Metodo que retorna todos os doc_ids em que a palavra ocorre
        return list(self.docs.keys())
    
    def getTf(self, doc_id) : # Metodo que retorna o tf da palavra em um documento, ou 0 caso não ocorra
        return self.docs.get(doc_id, 0)

In [21]:
tabela = pd.read_csv("estadao_noticias_eleicao.csv", encoding="utf-8")
tabela.fillna('', inplace=True) # Preenchendo os campos vazios da tabela com ''

In [33]:
mapa = {} # Dicionario/Mapa que representa o indice invertido

In [34]:
documentos = {} # Dicionario que guarda os documentos e seus tamanhos

In [44]:
for index, linha in tabela.iterrows() : # Iterando sobre as noticias no arquivo
    id = linha['idNoticia'] # Recuperando o idNoticia da noticia atual
    texto = nltk.wordpunct_tokenize(linha['titulo'] + ' ' + linha['subTitulo'] + ' ' + linha['conteudo']) # Tokenização do texto
    documentos[id] = len(texto) # Mapeamento do documento com seu tamanho
    for palavra in texto: # Iterando sobre as palavras na Noticia
        if (palavra not in string.punctuation): # Só ocorre o mapeamento se a palavra não é uma pontuação
            if (palavra.lower() not in mapa) : # Se a palavra ainda não foi mapeada, mapeia-a
                mapa[palavra.lower()] = WordInfo(palavra.lower())
            mapa[palavra.lower()].found(id) # Contabiliza a ocorrencia da palavra no documento atual

for k in mapa.keys() : # Laço para realizar o calculo dos idfs de cada palavra mapeada
    mapa[k].calculateIDF(len(documentos))

# Funções de Consultas Booleanas(And, Or e geral)

In [232]:
def searchAnd(palavra1, palavra2) :
    docs1 = mapa[palavra1.lower()].getIds() # Recupera todos as noticias em que "palavra1" ocorreu
    docs2 = mapa[palavra2.lower()].getIds() # Recupera todos as noticias em que "palavra2" ocorreu
    result = set() # Cria um conjunto vazio
    result.update(docs1) # Preenche o conjunto com os ids das noticias que "palavra1" aparece
    result = result.intersection(docs2) # Mantem no conjunto apenas os ids que as duas palavras ocorrem.
    return list(result)

In [233]:
def searchOr(palavra1, palavra2) :
    docs1 = mapa[palavra1.lower()].getIds() # Recupera todos as noticias em que "palavra1" ocorreu
    docs2 = mapa[palavra2.lower()].getIds() # Recupera todos as noticias em que "palavra2" ocorreu
    result = set() # Cria um conjunto vazio
    result.update(docs1) # Preenche o conjunto com os ids das noticias que "palavra1" aparece
    result.update(docs2) # Preenche o conjunto com os ids das noticias que "palavra2" aparece, por ser conjunto as duplicadas são eliminadas
    return list(result)

In [234]:
def search(consulta) :
    partes = consulta.split(' ')
    if (len(partes) < 2) : # Se a consulta só tem uma palavra
        return mapa[partes[0].lower()].getIds() # Recupera os ids que a palavra aparece
    elif (partes[1].upper() == 'AND') : # Se é uma consulta AND
        return searchAnd(partes[0], partes[2]) # Chama a função de consulta AND
    elif (partes[1].upper() == 'OR') : # Se é uma consulta OR
        return searchOr(partes[0], partes[2]) # Chama a função de consulta OR

# Funções de Consultas Vetoriais

In [74]:
def busca_binaria(consulta) :
    palavras = consulta.split(' ') # Quebrando a consulta em palavras
    relevant_docs = set(mapa[palavras[0]].getIds()) # Inicia o conjunto de documentos relevantes
    # Laço que realiza interseção dos conjuntos, para mantes apenas documentos em que todas as palavras a consulta ocorre
    for i in range(1, len(palavras)) :
        relevant_docs = relevant_docs.intersection(set(mapa[palavras[i]].getIds()))
    # Como é busca binaria, apenas retorna os primeiros 5 documentos que contem todas as palavras
    return list(relevant_docs)[:5]

In [75]:
def busca_tf(consulta) :
    palavras = consulta.split(' ') # Quebrando a consulta em palavras
    relevant_docs = set(mapa[palavras[0]].getIds()) # Inicia o conjunto de documentos relevantes
    # Laço que realiza interseção dos conjuntos, para mantes apenas documentos em que todas as palavras a consulta ocorre
    for i in range(1, len(palavras)) :
        relevant_docs = relevant_docs.intersection(set(mapa[palavras[i]].getIds()))
    result = [] # Lista que será gerado o resultado
    for doc_id in relevant_docs : # Para cada documento relevante
        scores = [mapa[w].getTf(doc_id) for w in palavras] # Obtem os tfs de cada palavra da consulta
        # Cria uma tupla (score, doc_id) somando os tfs obtidos e coloca na lista
        score_id = (sum(scores), doc_id)
        result.append(score_id)
    # Ordena do maior para o menor, considerando apenas o primeiro elemento da tupla para ordenar (o score)
    result = sorted(result, reverse=True, key=lambda tup: tup[0]) 
    result = [t[1] for t in result] # A lista passa a ser apenas dos doc_ids
    return result[:5] # Retorna os 5 primeiros

In [76]:
def busca_tfidf(consulta) :
    palavras = consulta.split(' ') # Quebrando a consulta em palavras
    relevant_docs = set(mapa[palavras[0]].getIds()) # Inicia o conjunto de documentos relevantes
    # Laço que realiza interseção dos conjuntos, para mantes apenas documentos em que todas as palavras a consulta ocorre
    for i in range(1, len(palavras)) :
        relevant_docs = relevant_docs.intersection(set(mapa[palavras[i]].getIds()))
    result = [] # Lista que será gerado o resultado
    for doc_id in relevant_docs : # Para cada documento relevante
        scores = [] # Lista para guardar os scores de cada palavra da consulta
        for w in palavras : # Calcula tf * idf de cada palavra da consulta e adiciona na lista
            m_palavra = mapa[w]
            w_score = m_palavra.getTf(doc_id) * m_palavra.idf
            scores.append(w_score)
        # Cria uma tupla (score, doc_id) somando os resultados obtidos e coloca na lista
        score_id = (sum(scores), doc_id)
        result.append(score_id)
    # Ordena do maior para o menor, considerando apenas o primeiro elemento da tupla para ordenar (o score)
    result = sorted(result, reverse=True, key=lambda tup: tup[0])
    result = [t[1] for t in result] # A lista passa a ser apenas dos doc_ids
    return result[:5] # Retorna os 5 primeiros

In [77]:
def busca_bm25(consulta) :
    k = 6 # 6 foi o valor de K que maximizou o resultado da função mapk
    palavras = consulta.split(' ') # Quebrando a consulta em palavras
    relevant_docs = set(mapa[palavras[0]].getIds()) # Inicia o conjunto de documentos relevantes
    # Laço que realiza interseção dos conjuntos, para mantes apenas documentos em que todas as palavras a consulta ocorre
    for i in range(1, len(palavras)) :
        relevant_docs = relevant_docs.intersection(set(mapa[palavras[i]].getIds()))
    result = [] # Lista que será gerado o resultado
    for doc_id in relevant_docs : # Para cada documento relevante
        scores = [] # Lista para guardar os scores de cada palavra da consulta
        for w in palavras : # Calcula tf*(k+1)/(tf + k) * idf de cada palavra da consulta e adiciona na lista
            m_palavra = mapa[w]
            tf = m_palavra.getTf(doc_id)
            w_score = (tf*(k+1))/(tf + k) * m_palavra.idf
            scores.append(w_score)
        # Cria uma tupla (score, doc_id) somando os resultados obtidos e coloca na lista
        score_id = (sum(scores), doc_id)
        result.append(score_id)
    # Ordena do maior para o menor, considerando apenas o primeiro elemento da tupla para ordenar (o score)
    result = sorted(result, reverse=True, key=lambda tup: tup[0])
    result = [t[1] for t in result] # A lista passa a ser apenas dos doc_ids
    return result[:5] # Retorna os 5 primeiros

# Avaliação Buscas Vetoriais

## Definição da função de comparação, Leitura do gabarito e realização das consultas

In [11]:
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [72]:
import ast
gabarito = pandas.read_csv('gabarito.csv')
# Transformando a string lida em lista de listas
gabarito_busca_binaria = list(gabarito.busca_binaria)
gabarito_tf = list(gabarito.tf)
gabarito_tfidf = list(gabarito.tfidf)
gabarito_bm25 = list(gabarito.bm25)
for i in range(len(gabarito_tf)) :
    gabarito_busca_binaria[i] = ast.literal_eval(gabarito_busca_binaria[i])
    gabarito_tf[i] = ast.literal_eval(gabarito_tf[i])
    gabarito_tfidf[i] = ast.literal_eval(gabarito_tfidf[i])
    gabarito_bm25[i] = ast.literal_eval(gabarito_bm25[i])

In [104]:
binaria_result, tf_result, tfidf_result, bm25_result = [], [], [], []
for consulta in gabarito.str_busca :
    binaria_result.append(busca_binaria(consulta))
    tf_result.append(busca_tf(consulta))
    tfidf_result.append(busca_tfidf(consulta))
    bm25_result.append(busca_bm25(consulta))

# Expansão de Consulta

Função de criação da matrix de termos-termos

In [48]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

In [49]:
content = tabela.titulo + " " + tabela.subTitulo + " " +  tabela.conteudo

In [50]:
tokens_lists = content.apply(lambda text: nltk.wordpunct_tokenize(text.lower()))

In [51]:
tokens = [token for tokens_list in tokens_lists for token in tokens_list]

In [52]:
matrix, vocab = co_occurrence_matrix(tokens)

In [54]:
consultable_matrix = matrix.tocsr()

In [55]:
def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

In [87]:
def top3_bigram(palavra) :
    bigram_freq = []
    for key in vocab.keys() :
        bigram_freq.append((consult_frequency(palavra, key), key))
    bigram_freq = sorted(bigram_freq, reverse=True)[:3]
    return [t[1] for t in bigram_freq]